In [1]:
from datetime import datetime as dt
import urllib
import pandas as pd
import ipywidgets as widgets

In [2]:
import pandas as pd
import ipywidgets as widgets
import requests
from IPython.display import display
import urllib.parse  
datasette_url = "https://datasette.planning.data.gov.uk/"

result_df = pd.DataFrame()

def update_dataframe(organisation):
    global result_df  
    if organisation:
        query = f" s.organisation = '{organisation}'"
    else:
        query = f" s.organisation LIKE '%'"
    params = urllib.parse.urlencode({
        "sql": f"""
        select
          e.endpoint_url,
          l.status,
          s.collection,
          group_concat(DISTINCT sp.pipeline) as pipelines,
          s.organisation,
          max(l.entry_date) maxentrydate,
          max(e.entry_date) entrydate,
          e.end_date
        from
          log l
          inner join source s on l.endpoint = s.endpoint
          inner join endpoint e on l.endpoint = e.endpoint
          inner join source_pipeline sp on s.source = sp.source
        where
           {query} and not collection="brownfield-land"
        group by
          l.endpoint,
          l.status
        order by
          l.endpoint,
          s.collection,
          maxentrydate desc
        """,
        "_size": "max"
    })
    
    url = f"{datasette_url}digital-land.csv?{params}"
    df = pd.read_csv(url)
    result_df = df
    return df
    
organisation_options = {
    "All":None,"Newcastle": "local-authority-eng:NET","Medway": "local-authority-eng:MDW","Lambeth": "local-authority-eng:LBH",
    "Gloucester": "local-authority-eng:GLO","Doncaster": "local-authority-eng:DNC","Buckinghamshire": "local-authority-eng:BUC","Epsom and Ewell": "local-authority-eng:EPS",
    "Canterbury": "local-authority-eng:CAT"
    
}

organisation_dropdown = widgets.Dropdown(
    options=organisation_options,
    description="Select LPA:",
)
def display_result_df():
    display(result_df)

widgets.interact(update_dataframe, organisation=organisation_dropdown)
initial_organisation = organisation_dropdown.value

interactive(children=(Dropdown(description='Select LPA:', options={'All': None, 'Newcastle': 'local-authority-…

In [6]:
new_df=result_df.copy()
new_df['maxentrydate'] = pd.to_datetime(new_df['maxentrydate'])
new_df['last_updated'] = None
new_df['last_status'] = None

for index, row in new_df.iterrows():
    if index < len(new_df) - 1 and (row['status'] == 404 or row['status'] == 500 or pd.isna(row['status'])):
        if row['endpoint_url'] == new_df.at[index + 1, 'endpoint_url']:
            new_df.at[index, 'last_updated'] = new_df.at[index + 1, 'maxentrydate']
            new_df.at[index, 'last_status'] = new_df.at[index + 1, 'status']
new_df.drop_duplicates(subset='endpoint_url', keep='first', inplace=True)
new_df.reset_index(drop=True, inplace=True)
print("\033[1m Endpoints with latest status: ")
new_df

 Endpoints with latest status: 


,endpoint_url,status,collection,pipelines,organisation,maxentrydate,entrydate,end_date,last_updated,last_status
0,https://datamillnorth.org/download/newcastle-a...,200,article-4-direction,article-4-direction-area,local-authority-eng:NET,2023-09-12 00:14:52+00:00,2023-02-23T22:22:12Z,NaN,None,None
1,https://datamillnorth.org/download/treepreserv...,200,tree-preservation-order,"tree,tree-preservation-order",local-authority-eng:NET,2023-09-12 00:15:59+00:00,2022-07-01T04:04:17Z,NaN,None,None
2,https://datamillnorth.org/download/listedbuild...,200,listed-building,listed-building-outline,local-authority-eng:NET,2023-09-12 00:14:58+00:00,2022-07-01T14:14:45Z,NaN,None,None
3,https://datamillnorth.org/download/treepreserv...,200,tree-preservation-order,"tree-preservation-order,tree-preservation-zone",local-authority-eng:NET,2023-09-12 00:15:59+00:00,2022-07-01T05:05:46Z,NaN,None,None
4,https://datamillnorth.org/download/newcastle-a...,200,article-4-direction,article-4-direction,local-authority-eng:NET,2023-09-12 00:14:52+00:00,2023-02-28T11:11:10Z,NaN,None,None
5,https://datamillnorth.org/download/conservatio...,200,conservation-area,conservation-area,local-authority-eng:NET,2023-09-12 00:04:38+00:00,2022-11-15T17:10:43Z,NaN,None,None
6,https://datamillnorth.org/download/conservatio...,404,conservation-area,conservation-area,local-authority-eng:NET,2023-09-12 00:04:38+00:00,2020-09-06T15:39:06Z,NaN,2022-06-06 00:05:42+00:00,200


In [7]:
filtered_df = new_df[new_df['status'] != 200]
filtered_df.reset_index(drop=True, inplace=True)
filtered_df

,endpoint_url,status,collection,pipelines,organisation,maxentrydate,entrydate,end_date,last_updated,last_status
0,https://datamillnorth.org/download/conservatio...,404,conservation-area,conservation-area,local-authority-eng:NET,2023-09-12 00:04:38+00:00,2020-09-06T15:39:06Z,NaN,2022-06-06 00:05:42+00:00,200


In [8]:
download = input("Do you want to download the results? (yes/no): ")

if download.lower() == "yes":
    # Save the DataFrame as a CSV file
    new_df.to_csv("endpoints_with_latest_status.csv", index=False)
    filtered_df.to_csv("endpoints_not_200.csv", index=False)
    print("Query result downloaded as 'endpoints_with_latest_status.csv'")
    print("Query result downloaded as 'endpoints_not_200.csv'")

Do you want to download the results? (yes/no):  yes


Query result downloaded as 'endpoints_with_latest_status.csv'
Query result downloaded as 'endpoints_not_200.csv'
